Для выполнения домашнего задания необходимо взять boston house-prices datase (sklearn.datasets.load_boston) и сделать тоже самое для задачи регрессии (попробовать разные алгоритмы, поподбирать параметры, вывести итоговое качество).

In [78]:
from sklearn import datasets
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso, Ridge, HuberRegressor, ElasticNet
from sklearn.tree import DecisionTreeRegressor
%matplotlib inline
import numpy as np
import pandas as pd
import random
from jupyterthemes import jtplot
jtplot.style()

In [22]:
# обеспечиваем воспроизводимость результата
random.seed(42)

In [4]:
boston = datasets.load_boston()
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [10]:
boston.data.shape

(506, 13)

In [72]:
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [13]:
X, y = boston['data'], boston['target']

In [18]:
# На валидацию откладываем 20%
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [19]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_valid = sc.transform(X_valid)

## Лассо регрессия

In [44]:
lasso_reg = Lasso()

In [55]:
# Для лассо только один параметр можно подобрать - альфа
lasso_params = {
    'alpha': np.logspace(-7, 2, 1000)
}
grid_lasso = GridSearchCV(lasso_reg, lasso_params, cv=10, verbose=2, n_jobs=-1)
grid_lasso.fit(X_train, y_train)

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done 268 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 3656 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 9340 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed:    7.2s finished
c:\venv\default\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': array([1.00000e-07, 1.02096e-07, ..., 9.79470e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [56]:
print(grid_lasso.best_params_)
print(grid_lasso.best_score_)
print(grid_lasso.best_estimator_)

{'alpha': 0.03195247505759213}
0.6923803081445976
Lasso(alpha=0.03195247505759213, copy_X=True, fit_intercept=True,
   max_iter=1000, normalize=False, positive=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False)


## Ридж-регрессия

In [58]:
rige_reg = Ridge()

In [59]:
# Для ридж-регрессии можно еще перебрать оптимизаторы
rige_params = {
    'alpha': np.logspace(-7, 2, 1000),
    'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}
grid_rige = GridSearchCV(rige_reg, rige_params, cv=10, verbose=2, n_jobs=-1)
grid_rige.fit(X_train, y_train)

Fitting 10 folds for each of 6000 candidates, totalling 60000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 3319 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 10627 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 20815 tasks      | elapsed:   29.2s
[Parallel(n_jobs=-1)]: Done 33955 tasks      | elapsed:   46.4s
[Parallel(n_jobs=-1)]: Done 49975 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 60000 out of 60000 | elapsed:  1.3min finished
c:\venv\default\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': array([1.00000e-07, 1.02096e-07, ..., 9.79470e+01, 1.00000e+02]), 'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [60]:
print(grid_rige.best_params_)
print(grid_rige.best_score_)
print(grid_rige.best_estimator_)

{'alpha': 12.826498305280625, 'solver': 'lsqr'}
0.6945198927331346
Ridge(alpha=12.826498305280625, copy_X=True, fit_intercept=True,
   max_iter=None, normalize=False, random_state=None, solver='lsqr',
   tol=0.001)


## Регрессия Хьюберта

In [64]:
huber_reg = HuberRegressor()

In [67]:
# Регрессия не шибко быстрая, поэтому сделаем параметров поменьше
huber_params = {
    'alpha': np.logspace(-7, 2, 100),
    'epsilon': np.linspace(1.35, 2, 50)
}
grid_huber = GridSearchCV(huber_reg, huber_params, cv=10, verbose=2, n_jobs=-1)
grid_huber.fit(X_train, y_train)

Fitting 10 folds for each of 5000 candidates, totalling 50000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 1107 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 3543 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 6939 tasks      | elapsed:   27.5s
[Parallel(n_jobs=-1)]: Done 11319 tasks      | elapsed:   43.5s
[Parallel(n_jobs=-1)]: Done 16659 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 22983 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 30267 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 38535 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 47763 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 50000 out of 50000 | elapsed:  3.0min finished
c:\venv\default\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
        tol=1e-05, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': array([1.00000e-07, 1.23285e-07, ..., 8.11131e+01, 1.00000e+02]), 'epsilon': array([1.35   , 1.36327, 1.37653, 1.3898 , 1.40306, 1.41633, 1.42959,
       1.44286, 1.45612, 1.46939, 1.48265, 1.49592, 1.50918, 1.52245,
       1.53571, 1.54898, 1.56224, 1.57551, 1.58878, 1.60204, 1...61, 1.89388,
       1.90714, 1.92041, 1.93367, 1.94694, 1.9602 , 1.97347, 1.98673,
       2.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [68]:
print(grid_huber.best_params_)
print(grid_huber.best_score_)
print(grid_huber.best_estimator_)

{'alpha': 0.1873817422860383, 'epsilon': 2.0}
0.6923587266712095
HuberRegressor(alpha=0.1873817422860383, epsilon=2.0, fit_intercept=True,
        max_iter=100, tol=1e-05, warm_start=False)


## ElasticNet регрессия
Как и регрессия Хьюберта объединяет l1 и l2 регуляризации

In [74]:
elast_reg = ElasticNet()

In [76]:
elast_params = {
    'alpha': np.logspace(-7, 2, 200),
    'l1_ratio': np.linspace(0, 1, 50)
}
grid_elast = GridSearchCV(elast_reg, elast_params, cv=10, verbose=2, n_jobs=-1)
grid_elast.fit(X_train, y_train)

Fitting 10 folds for each of 10000 candidates, totalling 100000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 2347 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 8843 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 17899 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done 29579 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done 43819 tasks      | elapsed:   47.8s
[Parallel(n_jobs=-1)]: Done 60683 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 80107 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 100000 out of 100000 | elapsed:  1.5min finished
c:\venv\default\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
c:\venv\default\lib\site-packages\sklearn\line

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': array([1.00000e-07, 1.10975e-07, ..., 9.01102e+01, 1.00000e+02]), 'l1_ratio': array([0.     , 0.02041, 0.04082, 0.06122, 0.08163, 0.10204, 0.12245,
       0.14286, 0.16327, 0.18367, 0.20408, 0.22449, 0.2449 , 0.26531,
       0.28571, 0.30612, 0.32653, 0.34694, 0.36735, 0.38776, ...33, 0.83673,
       0.85714, 0.87755, 0.89796, 0.91837, 0.93878, 0.95918, 0.97959,
       1.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [77]:
print(grid_elast.best_params_)
print(grid_elast.best_score_)
print(grid_elast.best_estimator_)

{'alpha': 0.040554607358408275, 'l1_ratio': 0.0}
0.6941843276314297
ElasticNet(alpha=0.040554607358408275, copy_X=True, fit_intercept=True,
      l1_ratio=0.0, max_iter=1000, normalize=False, positive=False,
      precompute=False, random_state=None, selection='cyclic', tol=0.0001,
      warm_start=False)


#### Судя по выбранному параметру l1_ratio, модель скатилась к l2-регуляризации

## На Кегле для этой задачи используют DecisionTree, пробуем

In [79]:
tree_reg = DecisionTreeRegressor()

In [80]:
tree_params = {
    'max_depth': range(1, 11),
    'splitter': ['best', 'random'],
    'criterion': ['mse', 'mae', 'friedman_mse'],
    'min_samples_leaf': [1, 2, 4, 8, 16]
}
grid_tree = GridSearchCV(tree_reg, tree_params, cv=10, verbose=2, n_jobs=-1)
grid_tree.fit(X_train, y_train)

Fitting 10 folds for each of 300 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 2514 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed:    5.8s finished
c:\venv\default\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': range(1, 11), 'splitter': ['best', 'random'], 'criterion': ['mse', 'mae', 'friedman_mse'], 'min_samples_leaf': [1, 2, 4, 8, 16]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [81]:
print(grid_tree.best_params_)
print(grid_tree.best_score_)
print(grid_tree.best_estimator_)

{'criterion': 'friedman_mse', 'max_depth': 6, 'min_samples_leaf': 2, 'splitter': 'best'}
0.7840973540990791
DecisionTreeRegressor(criterion='friedman_mse', max_depth=6,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')


#### На трейн данных пока лучше всех себя показало дерево

## Сравниваем на валидационной выборке

In [82]:
estimators = {
    'lasso': grid_lasso,
    'rige': grid_rige,
    'huber': grid_huber,
    'elasticNet': grid_elast,
    'tree': grid_tree
}

In [83]:
for k in estimators:
    v = estimators[k]
    print(k, "CV R^2:", v.best_score_, "Validation R^2:", v.best_estimator_.score(X_valid, y_valid))

lasso CV R^2: 0.6923803081445976 Validation R^2: 0.7399451118759672
rige CV R^2: 0.6945198927331346 Validation R^2: 0.7395326131234168
huber CV R^2: 0.6923587266712095 Validation R^2: 0.719390063552706
elasticNet CV R^2: 0.6941843276314297 Validation R^2: 0.740045591704626
tree CV R^2: 0.7840973540990791 Validation R^2: 0.837243343098626


#### Все модели показали на валидационной выборке результаты лучше, чем на тренировочной

#### Лучше всего показало себя дерево

## Смотрим внимательнее на результат

In [84]:
y_pred = grid_tree.best_estimator_.predict(X_valid)

In [106]:
compare = pd.DataFrame(y_pred, columns=['Предсказание'])
compare['Реальность'] = y_valid
compare['Разница'] = compare['Реальность'] - compare['Предсказание']
compare['Разница, %'] = 100*compare['Разница']/compare['Реальность']
compare.head(15)

,Предсказание,Реальность,Разница,"Разница, %"
0,24.519355,28.4,3.880645,13.664244
1,23.650000,27.5,3.850000,14.000000
2,24.519355,23.9,-0.619355,-2.591443
3,21.730000,19.5,-2.230000,-11.435897
4,20.697468,21.4,0.702532,3.282858
5,7.858333,8.7,0.841667,9.674330
6,45.950000,50.0,4.050000,8.100000
7,7.858333,7.2,-0.658333,-9.143519
8,33.653846,36.1,2.446154,6.776049
9,24.519355,23.7,-0.819355,-3.457193


In [108]:
(compare['Разница, %'].max(), compare['Разница, %'].min())

(33.29439252336448, -119.33333333333333)

#### Несмотря на хорошее значение R^2, отклонения от предсказания в конкретных случаях могут быть значительными